# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into Deep Neural Networks for NLP!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of Word2Vec such as vector size and window size

## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec Model. Then, you'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [2]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what your data looks like. 

In [4]:
raw_df = pd.read_json('News_Category_Dataset_v2.json', lines=True)

In [5]:
raw_df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data--`headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [11]:
pd.set_option('max_colwidth',500)

In [13]:
raw_df['combined_text'] = raw_df['headline']+' '+raw_df['short_description']
raw_df.head()

,authors,category,date,headline,link,short_description,combined_text
0,Melissa Jeltsen,CRIME,2018-05-26,"There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV",https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89,She left her husband. He killed their children. Just another day in America.,"There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV She left her husband. He killed their children. Just another day in America."
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song,https://www.huffingtonpost.com/entry/will-smith-joins-diplo-and-nicky-jam-for-the-official-2018-world-cup-song_us_5b09726fe4b0fdb2aa541201,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-grant-marries_us_5b09212ce4b0568a880b9a8c,The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony.,Hugh Grant Marries For The First Time At Age 57 The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony.
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork,https://www.huffingtonpost.com/entry/jim-carrey-adam-schiff-democrats_us_5b0950e8e4b0fdb2aa53e675,The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump.,Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump.
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog,https://www.huffingtonpost.com/entry/julianna-margulies-trump-poop-bag_us_5b093ec2e4b0fdb2aa53df70,"The ""Dietland"" actress said using the bags is a ""really cathartic, therapeutic moment.""","Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog The ""Dietland"" actress said using the bags is a ""really cathartic, therapeutic moment."""


In [18]:
data = list(map(lambda words: word_tokenize(words), raw_df['combined_text']))

Inspect the first 5 items in `data` to see how everything looks. 

In [19]:
data[:5]

[['There',
  'Were',
  '2',
  'Mass',
  'Shootings',
  'In',
  'Texas',
  'Last',
  'Week',
  ',',
  'But',
  'Only',
  '1',
  'On',
  'TV',
  'She',
  'left',
  'her',
  'husband',
  '.',
  'He',
  'killed',
  'their',
  'children',
  '.',
  'Just',
  'another',
  'day',
  'in',
  'America',
  '.'],
 ['Will',
  'Smith',
  'Joins',
  'Diplo',
  'And',
  'Nicky',
  'Jam',
  'For',
  'The',
  '2018',
  'World',
  'Cup',
  "'s",
  'Official',
  'Song',
  'Of',
  'course',
  'it',
  'has',
  'a',
  'song',
  '.'],
 ['Hugh',
  'Grant',
  'Marries',
  'For',
  'The',
  'First',
  'Time',
  'At',
  'Age',
  '57',
  'The',
  'actor',
  'and',
  'his',
  'longtime',
  'girlfriend',
  'Anna',
  'Eberstein',
  'tied',
  'the',
  'knot',
  'in',
  'a',
  'civil',
  'ceremony',
  '.'],
 ['Jim',
  'Carrey',
  'Blasts',
  "'Castrato",
  "'",
  'Adam',
  'Schiff',
  'And',
  'Democrats',
  'In',
  'New',
  'Artwork',
  'The',
  'actor',
  'gives',
  'Dems',
  'an',
  'ass-kicking',
  'for',
  'not',
 

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [20]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count`. 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [21]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55561945, 67352790)

Great! you now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which is stored inside of `model.wv`. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [22]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [23]:
wv.most_similar('Texas')

[('Pennsylvania', 0.8189535737037659),
 ('Illinois', 0.8165842294692993),
 ('Ohio', 0.8101603388786316),
 ('Maryland', 0.7988088130950928),
 ('Connecticut', 0.795230507850647),
 ('Oregon', 0.7915254831314087),
 ('Georgia', 0.7855269908905029),
 ('Massachusetts', 0.7853920459747314),
 ('California', 0.781449556350708),
 ('Oklahoma', 0.7793487906455994)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [24]:
wv.most_similar(negative='Texas')

[('much-vaunted', 0.46132025122642517),
 ('Per-Mile', 0.43481960892677307),
 ('Headstrong', 0.4114622473716736),
 ('Parent/Grandparent', 0.40511295199394226),
 ('Palatetraining', 0.3898324966430664),
 ('Fastest-Rising', 0.3811153173446655),
 ('Relational', 0.3711831569671631),
 ('Admon', 0.36504942178726196),
 ('Ex-Ambassador', 0.36273887753486633),
 ('Animal-rescue', 0.36119791865348816)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [25]:
wv['Texas']

array([-0.53794616, -1.3601084 ,  1.3117241 , -0.25871643,  0.27418166,
        0.32738122, -0.78228426,  0.71716505, -0.91824174, -1.3354577 ,
       -0.7916994 , -0.06932025,  0.5588611 ,  0.00705083,  1.6004928 ,
        1.5526143 , -2.6434937 , -1.1674179 ,  2.2259514 , -1.30209   ,
       -1.7455813 ,  2.021349  , -0.68356067,  0.35647696,  0.2856911 ,
        3.6068695 , -0.5979051 ,  1.0112783 ,  0.58780307, -0.7538051 ,
       -3.296302  , -1.717207  , -0.9430681 , -1.2916571 ,  2.4761605 ,
       -0.09439196, -0.5520367 ,  2.0272758 , -1.8373659 , -0.22312568,
       -1.0707976 ,  0.7388607 ,  0.8162085 , -1.0449818 , -1.2141532 ,
       -0.5714529 ,  0.66024107,  5.0850897 , -2.224582  ,  2.1558375 ,
        0.36311537,  0.8959992 ,  0.63599694,  1.0341178 , -0.00880119,
       -1.5151541 ,  2.03175   ,  1.278402  , -0.6412613 ,  1.7670667 ,
       -0.13484448, -0.605133  , -0.38899773,  2.0837991 , -0.46969777,
        0.07181405,  1.3659828 ,  1.3842101 ,  0.13974163, -0.31

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [26]:
wv.vectors

array([[-5.94964504e-01, -1.68747377e+00,  5.70918918e-01, ...,
        -1.29654124e-01,  3.73233944e-01, -9.91780400e-01],
       [-6.38495326e-01, -2.15257764e+00, -9.32521701e-01, ...,
         1.31358635e+00, -5.25853813e-01,  9.42604780e-01],
       [-2.10301623e-01, -9.10640597e-01, -1.47101998e-01, ...,
         2.34582424e-01,  2.03529835e+00,  1.59654975e-01],
       ...,
       [ 9.00428072e-02,  9.49495584e-02, -6.64059073e-02, ...,
         3.60338055e-02,  3.22833424e-04, -7.24223908e-03],
       [ 5.32997027e-02,  1.83119345e-02,  4.77484427e-02, ...,
         3.46508846e-02,  1.35143390e-02, -2.45101154e-02],
       [-2.33640317e-02, -6.55893460e-02, -3.41670290e-02, ...,
         7.12076053e-02, -1.13658225e-02, -3.31617929e-02]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `most_similar` function and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [28]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('princess', 0.6173090934753418),
 ('invention', 0.5841718316078186),
 ('queen', 0.565031886100769),
 ('revival', 0.5591679215431213),
 ('villain', 0.555040717124939),
 ('brunette', 0.5505114793777466),
 ('beast', 0.5415003299713135),
 ('seductive', 0.5395407676696777),
 ('title', 0.5390926599502563),
 ('staple', 0.5383564233779907)]

In [35]:
wv.most_similar(positive=['cat','pet','animal'])

[('dog', 0.7569730877876282),
 ('toy', 0.6961859464645386),
 ('puppy', 0.6664701700210571),
 ('pro', 0.6636679768562317),
 ('lover', 0.6492595672607422),
 ('stuffed', 0.6365921497344971),
 ('plant', 0.6300227642059326),
 ('toddler', 0.6247068643569946),
 ('wardrobe', 0.6185210943222046),
 ('vest', 0.615349292755127)]

As you can see from the output above, your model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As you can see from the word in first place, 'reminiscent', your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to created vectorized word embeddings!